In [107]:
import pandas as pd
from io import StringIO
import os

## Read trace

In [108]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

### COORD

In [109]:
nr_worker = 12
sub_dir = str(nr_worker) + "rm"
trace_dir = "/home/posheng/billy_ws/fl_025/fl_experiment_trace/nchc/" + sub_dir + "/psnet_18"
coord_trace_file = "coord.txt"

worker_files = []
for i in range(2, 2+nr_worker):
    worker_files.append("flvm-{}.txt".format(i))

# hpc0_trace_file = "hpc0.txt"
# hpc1_trace_file = "hpc1.txt"
# hpc2_trace_file = "hpc2.txt"
# hpc3_trace_file = "hpc3_trace.txt"

In [110]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

coord_df = read_trace(trace_dir, coord_trace_file)
# w0_df = read_trace(trace_dir, hpc0_trace_file, "alice")
# w1_df = read_trace(trace_dir, hpc1_trace_file, "bob")
# w2_df = read_trace(trace_dir, hpc2_trace_file, "charlie")

worker_dfs = {}
for i in range(len(worker_files)):
    worker_df = read_trace(trace_dir, worker_files[i], worker_files[i].split(".")[0])
    worker_dfs[worker_files[i].split(".")[0]] = worker_df

## Duration

In [111]:
def get_worker_duration(worker_id, worker_dfs, event_type):
    worker_df = worker_dfs[worker_id]
    worker_event_df = worker_df[worker_df["EventType"].str.contains(event_type, regex=False)]
    worker_event_df = worker_event_df.reset_index()
    
    del worker_event_df['index']
    del worker_event_df['ID']
    
    return worker_event_df

In [112]:
def get_coord_duration(coord_df, event_type):
    coord_event_df = coord_df[coord_df["EventType"].str.contains(event_type, regex=False)]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    del coord_event_df['ID']
    
    return coord_event_df

In [113]:
def calculate_duration(event_type, df):
    event_df = df[df["EventType"] == event_type][["EventType", "EventState", "ID", "Time"]]
    event_start_df = event_df[event_df["EventState"] == "start"][["ID", "Time"]]
    event_start_df = event_start_df.reset_index()

    event_end_df = event_df[event_df["EventState"] == "end"][["Time"]]
    event_end_df = event_end_df.reset_index()
    
    event_df_cat = pd.concat([event_start_df, event_end_df], axis=1)
    del event_df_cat['index']
    event_df_cat.columns = ["ID", "StartTime", "EndTime"]
    
    event_df_cat["Duration"] = event_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    event_df_cat.insert(0, "EventType", event_type)
    return event_df_cat


In [114]:
GlobalModelSendDuration = calculate_duration("GlobalModelSend", coord_df)

In [115]:
LossFuncSendDuration = calculate_duration("LossFuncSend", coord_df)

In [116]:
ModelConfigSendDuration = calculate_duration("ModelConfigSend", coord_df)

### GlobalModelSendDuration

In [117]:
for worker_id in worker_dfs.keys():
    gms_mean = GlobalModelSendDuration[GlobalModelSendDuration["ID"] == worker_id][1:]['Duration'].mean()
    print("GlobalModelSendDuration", worker_id, gms_mean)

GlobalModelSendDuration flvm-2 7.667613196372986
GlobalModelSendDuration flvm-3 7.564564967155457
GlobalModelSendDuration flvm-4 7.340527379512787
GlobalModelSendDuration flvm-5 7.530710542201996
GlobalModelSendDuration flvm-6 7.478069269657135
GlobalModelSendDuration flvm-7 7.19609557390213
GlobalModelSendDuration flvm-8 7.449051940441132
GlobalModelSendDuration flvm-9 7.649378037452697
GlobalModelSendDuration flvm-10 7.988063013553619
GlobalModelSendDuration flvm-11 7.513224577903747
GlobalModelSendDuration flvm-12 7.397032868862152
GlobalModelSendDuration flvm-13 7.418291425704956


### LossFuncSendDuration

In [118]:
for worker_id in worker_dfs.keys():
    lfs_mean = LossFuncSendDuration[LossFuncSendDuration["ID"] == worker_id][1:]['Duration'].mean()
    print("LossFuncSendDuration", worker_id, lfs_mean)

LossFuncSendDuration flvm-2 0.0020514011383056642
LossFuncSendDuration flvm-3 0.0013728737831115723
LossFuncSendDuration flvm-4 0.0014841556549072266
LossFuncSendDuration flvm-5 0.0014796257019042969
LossFuncSendDuration flvm-6 0.0016753196716308594
LossFuncSendDuration flvm-7 0.0013574123382568359
LossFuncSendDuration flvm-8 0.0018759965896606445
LossFuncSendDuration flvm-9 0.0015209078788757323
LossFuncSendDuration flvm-10 0.0017450809478759765
LossFuncSendDuration flvm-11 0.0014450907707214355
LossFuncSendDuration flvm-12 0.0016813278198242188
LossFuncSendDuration flvm-13 0.0020964622497558595


### ModelConfigSendDuration

In [119]:
for worker_id in worker_dfs.keys():
    mcs_mean = ModelConfigSendDuration[ModelConfigSendDuration["ID"] == worker_id][1:]['Duration'].mean()
    print("ModelConfigSendDuration", worker_id, mcs_mean)

ModelConfigSendDuration flvm-2 0.0009002804756164551
ModelConfigSendDuration flvm-3 0.001006174087524414
ModelConfigSendDuration flvm-4 0.0008531928062438965
ModelConfigSendDuration flvm-5 0.000994265079498291
ModelConfigSendDuration flvm-6 0.0011876583099365234
ModelConfigSendDuration flvm-7 0.0008446574211120605
ModelConfigSendDuration flvm-8 0.0009863853454589843
ModelConfigSendDuration flvm-9 0.0006768941879272461
ModelConfigSendDuration flvm-10 0.0008260488510131836
ModelConfigSendDuration flvm-11 0.0008796811103820801
ModelConfigSendDuration flvm-12 0.0009310960769653321
ModelConfigSendDuration flvm-13 0.00095595121383667


## GlobalInformationSend Duration

In [120]:
def get_coord_global_send_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("GlobalInformationSend", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [121]:
for worker_id in worker_dfs.keys():
    gs_df = get_coord_global_send_duration(coord_df, worker_id)
    print("GlobalInformationSend", worker_id, gs_df['Time'][1:].mean())

GlobalInformationSend flvm-2 7.684575390815735
GlobalInformationSend flvm-3 7.478808724880219
GlobalInformationSend flvm-4 7.486561417579651
GlobalInformationSend flvm-5 7.617390298843384
GlobalInformationSend flvm-6 7.491522705554962
GlobalInformationSend flvm-7 6.953656399250031
GlobalInformationSend flvm-8 7.3916141271591185
GlobalInformationSend flvm-9 7.510558927059174
GlobalInformationSend flvm-10 8.240407633781434
GlobalInformationSend flvm-11 7.765078496932984
GlobalInformationSend flvm-12 7.200858151912689
GlobalInformationSend flvm-13 7.402940428256988


## Send Time

In [122]:
## EventType:
##     * GlobalModelSend
##     * LossFuncSend
##     * ModelConfigSend
def calculate_send_time(coord_df, worker_dfs, event_type):
    event_start_df = coord_df[(coord_df["EventType"] == event_type) & (coord_df["EventState"] == "start")]
    worker_event_send_time_dfs = {}
    
    for worker_id in worker_dfs.keys():
        worker_event_start_df = event_start_df[event_start_df["ID"] == worker_id]
        worker_event_start_df = worker_event_start_df.reset_index()
        del worker_event_start_df['index']
        worker_event_start_df.columns = ["EventType", "EventState", "ID", "StartTime"]
        
        worker_df = worker_dfs[worker_id]
        worker_event_recv_df = worker_df[worker_df["EventType"] == event_type]
        worker_event_recv_df = worker_event_recv_df.reset_index()
        del worker_event_recv_df['index']
        worker_event_recv_df.columns = ["WorkerID", "EventType", "EventState", "ID", "RecvTime"]
        
        worker_recv_time_series = worker_event_recv_df["RecvTime"]
        worker_event_send_time_df = pd.concat([worker_event_start_df, worker_recv_time_series], axis=1)
        worker_event_send_time_df["SendTime"] = worker_event_send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
        del worker_event_send_time_df["EventState"]
        
        worker_event_send_time_dfs[worker_id] = worker_event_send_time_df
        
    return worker_event_send_time_dfs

In [123]:
# worker_gms_send_time_dfs = calculate_send_time(coord_df, worker_dfs, "GlobalModelSend")

In [124]:
# worker_gms_send_time_dfs['alice']['SendTime'].mean()

In [125]:
# worker_gms_send_time_dfs['alice']

In [126]:
# worker_gms_send_time_dfs['charlie']['SendTime'].mean()

In [127]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

## LocalTraining Duration

In [128]:
for worker_id in worker_dfs.keys():
    lt_df = get_worker_duration(worker_id, worker_dfs, "LocalTraining")
    print("LocalTrainingDuration", worker_id, lt_df['Time'][1:].mean())

LocalTrainingDuration flvm-2 14.527965557575225
LocalTrainingDuration flvm-3 20.60708519220352
LocalTrainingDuration flvm-4 13.968101966381074
LocalTrainingDuration flvm-5 15.080920398235321
LocalTrainingDuration flvm-6 18.214482486248016
LocalTrainingDuration flvm-7 12.160878157615661
LocalTrainingDuration flvm-8 15.166603779792785
LocalTrainingDuration flvm-9 18.18548139333725
LocalTrainingDuration flvm-10 12.551630294322967
LocalTrainingDuration flvm-11 16.007501304149628
LocalTrainingDuration flvm-12 19.544489133358002
LocalTrainingDuration flvm-13 14.986537373065948


## FitSagg Duration

In [129]:
def get_coord_fitsagg_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("FitSagg", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [130]:
for worker_id in worker_dfs.keys():
    fs_df = get_coord_fitsagg_duration(coord_df, worker_id)
    print("FitSaggDuration", worker_id, fs_df['Time'][1:].mean())

FitSaggDuration flvm-2 344.3517149567604
FitSaggDuration flvm-3 341.0779214501381
FitSaggDuration flvm-4 337.4497375488281
FitSaggDuration flvm-5 337.26308304071426
FitSaggDuration flvm-6 338.0131596684456
FitSaggDuration flvm-7 337.6765790462494
FitSaggDuration flvm-8 340.7274755716324
FitSaggDuration flvm-9 339.9393992185593
FitSaggDuration flvm-10 341.5855336070061
FitSaggDuration flvm-11 343.0197478055954
FitSaggDuration flvm-12 343.2106821775436
FitSaggDuration flvm-13 343.2384354233742


## EncryptMultiply Duration

In [131]:
for worker_id in worker_dfs.keys():
    em_df = get_worker_duration(worker_id, worker_dfs, "EncryptMultiply")
    print("EncryptMultiplyDuration", worker_id, em_df['Time'][1:].mean())

EncryptMultiplyDuration flvm-2 320.69002866744995
EncryptMultiplyDuration flvm-3 311.44677129983904
EncryptMultiplyDuration flvm-4 314.1710296034813
EncryptMultiplyDuration flvm-5 312.9767508506775
EncryptMultiplyDuration flvm-6 310.6534842014313
EncryptMultiplyDuration flvm-7 316.5576479554176
EncryptMultiplyDuration flvm-8 316.4881857872009
EncryptMultiplyDuration flvm-9 312.82067815065386
EncryptMultiplyDuration flvm-10 319.56651624441145
EncryptMultiplyDuration flvm-11 317.62938759326937
EncryptMultiplyDuration flvm-12 314.5029323339462
EncryptMultiplyDuration flvm-13 319.10074441432954


## MultiplyParameter Duration

In [133]:
def get_multiply_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('MultiplyParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df

In [152]:
for worker_id in worker_dfs.keys():
    mul_df = get_worker_duration(worker_id, worker_dfs, "MultiplyParameter")
    start_indices = list(mul_df[mul_df["EventType"] == "MultiplyParameter0"].index)
    
    multime_list = []
    for start_index in start_indices:
        multime = mul_df[start_index:start_index+10]["Time"].sum()
        multime_list.append(multime)
    print("MultiplyParameter", sum(multime_list) / len(multime_list))

MultiplyParameter 79.57001128650847
MultiplyParameter 65.53368283453442
MultiplyParameter 61.012371528716315
MultiplyParameter 60.71301301320394
MultiplyParameter 68.67875796272641
MultiplyParameter 70.11277345248631
MultiplyParameter 64.23990440368652
MultiplyParameter 76.55564148085458
MultiplyParameter 67.41831664811997
MultiplyParameter 60.82816811970302
MultiplyParameter 57.50939080828712
MultiplyParameter 55.98325829278855


In [134]:
worker_dfs.keys()

dict_keys(['flvm-2', 'flvm-3', 'flvm-4', 'flvm-5', 'flvm-6', 'flvm-7', 'flvm-8', 'flvm-9', 'flvm-10', 'flvm-11', 'flvm-12', 'flvm-13'])

In [153]:
mul_df = get_worker_duration('flvm-3', worker_dfs, "MultiplyParameter")

In [154]:
start_indices = list(mul_df[mul_df["EventType"] == "MultiplyParameter0"].index)

In [155]:
start_indices[2]

20

In [156]:
multime = mul_df[20:20+10]["Time"].sum()

In [160]:
mul_df[40:40+10]

,WorkerID,EventType,EventState,Time
40,flvm-3,MultiplyParameter0,duration,0.014828
41,flvm-3,MultiplyParameter1,duration,0.012037
42,flvm-3,MultiplyParameter2,duration,0.083364
43,flvm-3,MultiplyParameter3,duration,2.173817
44,flvm-3,MultiplyParameter4,duration,14.265539
45,flvm-3,MultiplyParameter5,duration,7.227415
46,flvm-3,MultiplyParameter6,duration,9.680954
47,flvm-3,MultiplyParameter7,duration,5.229823
48,flvm-3,MultiplyParameter8,duration,4.207125
49,flvm-3,MultiplyParameter9,duration,4.457625


In [87]:
# # hpc1_mp_df = get_multiply_parameter_duration("alice", worker_dfs)
# hpc1_mp_df = get_worker_duration("alice", worker_dfs, "MultiplyParameter")
# hpc2_mp_df = get_worker_duration("bob", worker_dfs, "MultiplyParameter")
# hpc3_mp_df = get_worker_duration("charlie", worker_dfs, "MultiplyParameter")

In [88]:
# hpc1_mp_df[:8]

In [89]:
# hpc1_mp_df[:8]["Time"].sum()

## RetrieveTime duration

In [90]:
def get_coord_retrieve_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("RetrieveTime", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [91]:
for worker_id in worker_dfs.keys():
    ret_df = get_coord_retrieve_duration(coord_df, worker_id)
    print("RetrieveTimeDuration", worker_id, ret_df["Time"][1:].mean())

RetrieveTimeDuration flvm-2 0.014584219455718994
RetrieveTimeDuration flvm-3 0.016673183441162108
RetrieveTimeDuration flvm-4 0.01969914436340332
RetrieveTimeDuration flvm-5 0.016743409633636474
RetrieveTimeDuration flvm-6 0.015228724479675293
RetrieveTimeDuration flvm-7 0.014920151233673096
RetrieveTimeDuration flvm-8 0.017406320571899413
RetrieveTimeDuration flvm-9 0.016465139389038087
RetrieveTimeDuration flvm-10 0.015398812294006348
RetrieveTimeDuration flvm-11 0.014381778240203858
RetrieveTimeDuration flvm-12 0.015300965309143067
RetrieveTimeDuration flvm-13 0.014986717700958252


## ShareSend Duration

In [92]:
def calculate_share_send_duration(worker_id, worker_dfs, nr_parameters):
    worker_df = worker_dfs[worker_id]
    worker_send_share_df = worker_df[worker_df["EventType"].str.contains('SendShare', regex=False)]
    
    worker_send_share_start_df = worker_send_share_df[worker_send_share_df["EventState"] == 'start']
    worker_send_share_start_df = worker_send_share_start_df.reset_index()
    worker_send_share_start_df = worker_send_share_start_df[["WorkerID", "EventType", "ID", "Time"]]
    worker_send_share_start_df.columns = ["WorkerID", "EventType", "ID", "StartTime"]
    
    worker_send_share_end_df = worker_send_share_df[worker_send_share_df["EventState"] == 'end']
    worker_send_share_end_df = worker_send_share_end_df.reset_index()
    
    send_share_df_cat = pd.concat([worker_send_share_start_df, worker_send_share_end_df["Time"]], axis=1)
    send_share_df_cat.columns = ["WorkerID", "EventType", "ID", "StartTime", "EndTime"]
    send_share_df_cat["Duration"] = send_share_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    
    parameter_index = []
    nr_worker = len(worker_dfs)
    p_index = -1

    for i in range(len(send_share_df_cat)):
        tmp_index = (i%nr_worker)
        if tmp_index == 0:
            p_index += 1
            p_index = p_index % nr_parameters
        parameter_index.append(p_index)
        
    send_share_df_cat.insert(2, "p_idx", parameter_index)
    obj_id_list = send_share_df_cat.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_share_df_cat.insert(2, "obj_id", obj_id_list)
    
    del send_share_df_cat["EventType"]
    send_share_df_cat.insert(1, "EventType", "SendShare")
    
    return send_share_df_cat


In [93]:
# alice_send_share_df_cat = calculate_share_send_duration("alice", worker_dfs, 8)
# bob_send_share_df_cat = calculate_share_send_duration("bob", worker_dfs, 8)
# charlie_send_share_df_cat = calculate_share_send_duration("charlie", worker_dfs, 8)

## ShareSend Send Time

In [94]:
def calculate_share_send_time(sender_id, receiver_id, worker_dfs, nr_parameter):
    sender_df = worker_dfs[sender_id]
    receiver_df = worker_dfs[receiver_id]
    
    sender_send_share_df = sender_df[sender_df["EventType"].str.contains('SendShare', regex=False)]
    receiver_get_share_df = receiver_df[receiver_df["EventType"].str.contains('GetShare', regex=False)]
    
    send_share_start_df = sender_send_share_df[(sender_send_share_df["ID"] == receiver_id) & (sender_send_share_df["EventState"] == 'start')]
    get_share_df = receiver_get_share_df[receiver_get_share_df["ID"] == sender_id]
    send_share_start_df = send_share_start_df.reset_index()
    get_share_df = get_share_df.reset_index()
    
    del send_share_start_df['index']
    del send_share_start_df['EventState']
    
    send_time_df = pd.concat([send_share_start_df, get_share_df["Time"]], axis=1)
    send_time_df.columns = ["WorkerID", "EventType", "ID", "StartTime", "RecvTime"]
    send_time_df["SendTime"] = send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
    
    nr_parameter = 8
    parameter_index = []

    for i in range(len(send_time_df)):
        tmp_index = (i%nr_parameter)
        parameter_index.append(tmp_index)
        
    send_time_df.insert(2, "p_idx", parameter_index)
    
    obj_id_list = send_time_df.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_time_df.insert(2, "obj_id", obj_id_list)
    
    del send_time_df["EventType"]
    send_time_df.insert(1, "EventType", "SendShare")
    
    return send_time_df

In [95]:
# send_time_df = calculate_share_send_time("alice", "bob", worker_dfs, 8)

## AllWorkersTrainingTime Duration

In [96]:
coord_awtt_df = get_coord_duration(coord_df, "AllWorkersTrainingTime")

In [97]:
coord_awtt_df['Time'][1:].mean()

344.49829359054564

In [98]:
coord_awtt_df

,EventType,EventState,Time
0,AllWorkersTrainingTime,duration,341.686553
1,AllWorkersTrainingTime,duration,344.912729
2,AllWorkersTrainingTime,duration,338.530960
3,AllWorkersTrainingTime,duration,351.730963
4,AllWorkersTrainingTime,duration,340.165231
5,AllWorkersTrainingTime,duration,353.559435
6,AllWorkersTrainingTime,duration,343.004569
7,AllWorkersTrainingTime,duration,348.142894
8,AllWorkersTrainingTime,duration,344.046976
9,AllWorkersTrainingTime,duration,340.914059


## AggregationTime

In [99]:
coord_aggt_df = get_coord_duration(coord_df, "AggregationTime")

In [100]:
coord_aggt_df["Time"][1:].mean()

5.619293081760406

In [101]:
coord_aggt_df

,EventType,EventState,Time
0,AggregationTime,duration,7.071469
1,AggregationTime,duration,5.491187
2,AggregationTime,duration,5.309349
3,AggregationTime,duration,5.624215
4,AggregationTime,duration,5.516984
5,AggregationTime,duration,6.238954
6,AggregationTime,duration,5.858265
7,AggregationTime,duration,6.059559
8,AggregationTime,duration,5.730962
9,AggregationTime,duration,5.468255


## DecryptionTime

In [102]:
coord_decrypt_df = get_coord_duration(coord_df, "DecryptionTime")

In [103]:
coord_decrypt_df['Time'][1:].mean()

10.202981901168823

In [104]:
coord_decrypt_df

,EventType,EventState,Time
0,DecryptionTime,duration,12.183914
1,DecryptionTime,duration,9.268958
2,DecryptionTime,duration,9.954067
3,DecryptionTime,duration,10.254532
4,DecryptionTime,duration,10.850176
5,DecryptionTime,duration,10.641851
6,DecryptionTime,duration,9.928556
7,DecryptionTime,duration,9.414763
8,DecryptionTime,duration,10.466369
9,DecryptionTime,duration,10.453001


## RoundTime

In [105]:
coord_round_time_df = get_coord_duration(coord_df, "RoundTime")

In [106]:
coord_round_time_df["Time"].mean()

361.98342418670654

In [53]:
coord_round_time_df

,EventType,EventState,Time
0,RoundTime,duration,360.942077
1,RoundTime,duration,361.252839
2,RoundTime,duration,355.425129
3,RoundTime,duration,369.375824
4,RoundTime,duration,358.250212
5,RoundTime,duration,372.122801
6,RoundTime,duration,360.545349
7,RoundTime,duration,365.255847
8,RoundTime,duration,361.979611
9,RoundTime,duration,358.448572
